<a href="https://colab.research.google.com/github/JuanCervigon/Globalizacion/blob/main/Agrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

A partir de la tabla de Figaro en formato largo csv, se agregan ramas de actividad y se guarda el resultado en un excel en formato plano y otro en formato matricial

In [ ]:
# Importar librerías necesarias

import pandas as pd

# Importar datos

Importar los datos (drag and drop)

* Mapeo de los agregados originales a los finales

* Tabla flat Figaro

# Mapeo de los agregados

Crear un diccionario con mapeo de los agregados originales a los agregados definidos

* Leer un excel con el mapeo de los valores originales a los valores agregados y guardarlo como un DataFrame
* Convertir el DataFrame en un diccionario

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Leer el excel con el mapeo
mapeo=pd.read_excel('/content/drive/MyDrive/Figaro/mapeo.xlsx').set_index('original')
mapeo

,A10
original,
A01,A
A02,A
A03,A
B,BDE
C10T12,C
...,...
P3_S13,P3_S13
P3_S14,P3_S14
P3_S15,P3_S15


In [ ]:
# Convertir el DataFrame en un diccionario
colapsa=dict(mapeo.to_dict()['A10'])
colapsa

{'A01': 'A',
 'A02': 'A',
 'A03': 'A',
 'B': 'BDE',
 'C10T12': 'C',
 'C13T15': 'C',
 'C16': 'C',
 'C17': 'C',
 'C18': 'C',
 'C19': 'C',
 'C20': 'C',
 'C21': 'C',
 'C22': 'C',
 'C23': 'C',
 'C24': 'C',
 'C25': 'C',
 'C26': 'C',
 'C27': 'C',
 'C28': 'C',
 'C29': 'C',
 'C30': 'C',
 'C31_32': 'C',
 'C33': 'C',
 'D35': 'BDE',
 'E36': 'BDE',
 'E37T39': 'BDE',
 'F': 'F',
 'G45': 'GHI',
 'G46': 'GHI',
 'G47': 'GHI',
 'H49': 'GHI',
 'H50': 'GHI',
 'H51': 'GHI',
 'H52': 'GHI',
 'H53': 'GHI',
 'I': 'GHI',
 'J58': 'J',
 'J59_60': 'J',
 'J61': 'J',
 'J62_63': 'J',
 'K64': 'K',
 'K65': 'K',
 'K66': 'K',
 'L': 'L',
 'M69_70': 'MNOPQ',
 'M71': 'MNOPQ',
 'M72': 'MNOPQ',
 'M73': 'MNOPQ',
 'M74_75': 'MNOPQ',
 'N77': 'MNOPQ',
 'N78': 'MNOPQ',
 'N79': 'MNOPQ',
 'N80T82': 'MNOPQ',
 'O84': 'MNOPQ',
 'P85': 'MNOPQ',
 'Q86': 'MNOPQ',
 'Q87_88': 'MNOPQ',
 'R90T92': 'RSTU',
 'R93': 'RSTU',
 'S94': 'RSTU',
 'S95': 'RSTU',
 'S96': 'RSTU',
 'T': 'RSTU',
 'U': 'RSTU',
 'D21X31': 'D21X31',
 'OP_NRES': 'OP_NRES',
 'OP

# Crear una tabla Figaro colapsada


* Leer la tabla figaro en formato flat.
* Crear la columna `agrega`.
* Sumar `obsValue` colapsando `agrega`
* Separar la columna `agrega` en `origen` y `destino`
* Guardar la tabla
* Crear una tabla en formato matriz
* Salvar el resultado en excel


In [ ]:
# Leer el fichero Figaro flat en un dataframe [9363300 rows × 7 columns]
ruta='/content/drive/MyDrive/Figaro/flatfile_eu-ic-io_ind-by-ind_23ed_2021.csv'
df=pd.read_csv(ruta)
df

,icioiRow,icioiCol,refArea,rowIi,counterpartArea,colIi,obsValue
0,AR_A01,AR_A01,AR,A01,AR,A01,9811.568
1,AR_A02,AR_A01,AR,A02,AR,A01,73.217
2,AR_A03,AR_A01,AR,A03,AR,A01,12.623
3,AR_B,AR_A01,AR,B,AR,A01,491.892
4,AR_C10T12,AR_A01,AR,C10T12,AR,A01,1488.984
...,...,...,...,...,...,...,...
9363295,W2_OP_RES,ZA_P5M,W2,OP_RES,ZA,P5M,0.000
9363296,W2_OP_NRES,ZA_P5M,W2,OP_NRES,ZA,P5M,0.000
9363297,W2_D1,ZA_P5M,W2,D1,ZA,P5M,0.000
9363298,W2_D29X39,ZA_P5M,W2,D29X39,ZA,P5M,0.000


## Crear la columna `agregar`

In [ ]:
# Crear la columna agregar
df['agrega']=df.refArea+'_'+df.rowIi.map(colapsa)+\
  '*'+\
  df.counterpartArea+'_'+df.colIi.map(colapsa)

In [ ]:
df

,icioiRow,icioiCol,refArea,rowIi,counterpartArea,colIi,obsValue,agrega
0,AR_A01,AR_A01,AR,A01,AR,A01,9811.568,AR_A*AR_A
1,AR_A02,AR_A01,AR,A02,AR,A01,73.217,AR_A*AR_A
2,AR_A03,AR_A01,AR,A03,AR,A01,12.623,AR_A*AR_A
3,AR_B,AR_A01,AR,B,AR,A01,491.892,AR_BDE*AR_A
4,AR_C10T12,AR_A01,AR,C10T12,AR,A01,1488.984,AR_C*AR_A
...,...,...,...,...,...,...,...,...
9363295,W2_OP_RES,ZA_P5M,W2,OP_RES,ZA,P5M,0.000,W2_OP_RES*ZA_P5M
9363296,W2_OP_NRES,ZA_P5M,W2,OP_NRES,ZA,P5M,0.000,W2_OP_NRES*ZA_P5M
9363297,W2_D1,ZA_P5M,W2,D1,ZA,P5M,0.000,W2_D1*ZA_P5M
9363298,W2_D29X39,ZA_P5M,W2,D29X39,ZA,P5M,0.000,W2_D29X39*ZA_P5M


## Colapsar

Groupby devuelve una serie que hay que convertir en un DataFrame

In [ ]:
# Crear un dataframe con el campo obsValue colapsado
df=pd.DataFrame(df.groupby(['agrega'])['obsValue'].sum()).reset_index()

df

,agrega,obsValue
0,AR_A*AR_A,10228.996
1,AR_A*AR_BDE,45.784
2,AR_A*AR_C,26085.733
3,AR_A*AR_F,52.524
4,AR_A*AR_GHI,1776.859
...,...,...
321535,ZA_RSTU*ZA_P3_S14,10661.051
321536,ZA_RSTU*ZA_P3_S15,1127.947
321537,ZA_RSTU*ZA_P51G,5.276
321538,ZA_RSTU*ZA_P5M,0.017


## Crear las columnas `origen` y `destino`

In [ ]:
# Volver a crear las columnas origen y destion a partir de la columna agrega
df['origen']=df['agrega'].str.split('*').str[0]
df['destino']=df['agrega'].str.split('*').str[1]
df

,agrega,obsValue,origen,destino
0,AR_A*AR_A,10228.996,AR_A,AR_A
1,AR_A*AR_BDE,45.784,AR_A,AR_BDE
2,AR_A*AR_C,26085.733,AR_A,AR_C
3,AR_A*AR_F,52.524,AR_A,AR_F
4,AR_A*AR_GHI,1776.859,AR_A,AR_GHI
...,...,...,...,...
321535,ZA_RSTU*ZA_P3_S14,10661.051,ZA_RSTU,ZA_P3_S14
321536,ZA_RSTU*ZA_P3_S15,1127.947,ZA_RSTU,ZA_P3_S15
321537,ZA_RSTU*ZA_P51G,5.276,ZA_RSTU,ZA_P51G
321538,ZA_RSTU*ZA_P5M,0.017,ZA_RSTU,ZA_P5M


## Salvar la tabla larga en excel

In [ ]:
df.to_excel('/content/drive/MyDrive/Figaro/flat_agregado.xlsx',index=False)

# Formatear la tabla flat en una matriz

El dataset está en formato largo, es decir, un registro un valor. Un origen-destino un valor

El objetivo es reformatear el dataset a un registro por origen

In [ ]:
# Pasar a formato matriz 452 rows × 450 columns
df = df.pivot_table(index="origen", columns="destino", values="obsValue")
df

destino,AR_A,AR_BDE,AR_C,AR_F,AR_GHI,AR_J,AR_K,AR_L,AR_MNOPQ,AR_P3_S13,...,ZA_J,ZA_K,ZA_L,ZA_MNOPQ,ZA_P3_S13,ZA_P3_S14,ZA_P3_S15,ZA_P51G,ZA_P5M,ZA_RSTU
origen,,,,,,,,,,,,,,,,,,,,,
AR_A,10228.996,45.784,26085.733,52.524,1776.859,19.383,6.582,24.788,612.358,1511.112,...,0.040,0.000,0.013,0.411,0.000,7.160,0.000,0.000,0.000,0.044
AR_BDE,1078.689,8726.730,18414.884,1367.472,2121.975,353.362,163.017,706.098,2556.219,2534.637,...,0.002,0.000,0.002,0.011,0.000,0.007,0.000,0.000,0.000,0.001
AR_C,5623.285,4000.520,47582.160,9358.187,11697.838,2216.094,560.218,1455.504,6459.058,2284.889,...,1.677,0.019,0.464,15.973,0.000,105.331,0.000,2.880,0.000,1.201
AR_F,15.537,34.377,24.952,131.240,66.798,18.657,11.084,202.160,124.020,156.926,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
AR_GHI,3406.262,1630.726,17536.678,2147.322,9759.919,1734.937,565.136,515.491,4515.968,6560.812,...,0.385,0.017,0.089,1.773,0.017,12.271,0.000,0.242,-0.001,0.171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZA_J,0.006,0.036,0.099,0.011,0.183,0.578,0.157,0.021,0.399,0.274,...,5699.196,533.957,206.074,3980.551,4.605,8026.893,8.697,142.518,1.584,133.489
ZA_K,0.088,0.154,0.288,0.049,0.409,0.108,0.800,0.760,0.476,0.380,...,746.884,20242.640,4196.532,3613.475,0.005,12397.578,0.000,2.828,2.342,202.897
ZA_L,0.003,0.009,0.027,0.006,0.157,0.030,0.021,0.033,0.116,0.234,...,882.626,709.739,1572.076,3534.148,65.024,20259.080,0.000,448.771,0.052,332.348


## Salvar la tabla matriz en un excel

In [ ]:
df.to_excel('/content/drive/MyDrive/Figaro/matrix_agregado.xlsx')